## 1. Setup & Check GPU

In [ ]:
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ No GPU! Go to Runtime → Change runtime type → GPU")

In [ ]:
!pip install -q segmentation-models-pytorch albumentations opencv-python-headless

## 2. Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 3. Configuration

In [ ]:
# ============ CONFIG ============
import os

# Dataset paths - MASKS ARE ALREADY CREATED IN segmentation2 folder
DATASET_ROOT = "/content/drive/MyDrive/segmentation2"

# Training settings
ENCODER = 'resnet34'
ENCODER_WEIGHTS = 'imagenet'
EPOCHS = 60
IMG_SIZE = 640
BATCH_SIZE = 8
LEARNING_RATE = 0.0001
PATIENCE = 20

# Class names
CLASS_NAMES = [
    'background',
    'Longitudinal_crack',
    'Transverse_crack',
    'Alligator_crack',
    'Other_damage',
    'Pothole'
]
NUM_CLASSES = len(CLASS_NAMES)

# Output directory
OUTPUT_DIR = "/content/unet_outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Model: U-Net with {ENCODER} encoder")
print(f"Epochs: {EPOCHS}, Batch: {BATCH_SIZE}, ImgSize: {IMG_SIZE}")
print(f"Classes: {NUM_CLASSES} - {CLASS_NAMES}")

In [ ]:
# 3.1 Set random seed for reproducibility
import random
import numpy as np
import torch
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
set_seed(42)
print("Random seed set.")

## 4. Verify Dataset Structure

In [ ]:
print("📂 Checking dataset structure...\n")

for split in ['train', 'val', 'test']:
    images_dir = os.path.join(DATASET_ROOT, split, 'images')
    masks_dir = os.path.join(DATASET_ROOT, split, 'masks')
    
    if os.path.exists(images_dir) and os.path.exists(masks_dir):
        n_images = len([f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.png', '.jpeg'))])
        n_masks = len([f for f in os.listdir(masks_dir) if f.endswith('.png')])
        print(f"{split.upper()}:")
        print(f"  Images: {n_images}")
        print(f"  Masks:  {n_masks}")
        print()
    else:
        print(f"⚠️ {split.upper()}: NOT FOUND")
        print()

print("✓ Dataset verification complete!")

## 5. Create Dataset and DataLoader

In [ ]:
import torch
import numpy as np
import cv2
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2

class RoadDamageDataset(Dataset):
    def __init__(self, images_dir, masks_dir, transform=None):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.transform = transform
        self.images = sorted([f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.png', '.jpeg'))])
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.images_dir, img_name)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        mask_name = os.path.splitext(img_name)[0] + '.png'
        mask_path = os.path.join(self.masks_dir, mask_name)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        
        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']
        
        return image, mask.long()

# Transforms
train_transform = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.2),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=15, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.2),
    A.Blur(blur_limit=3, p=0.1),
    A.Cutout(num_holes=8, max_h_size=32, max_w_size=32, fill_value=0, p=0.5),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
], p=1.0)

val_transform = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
], p=1.0)

# Create datasets and loaders
train_dataset = RoadDamageDataset(
    os.path.join(DATASET_ROOT, 'train', 'images'),
    os.path.join(DATASET_ROOT, 'train', 'masks'),
    transform=train_transform
)

val_dataset = RoadDamageDataset(
    os.path.join(DATASET_ROOT, 'val', 'images'),
    os.path.join(DATASET_ROOT, 'val', 'masks'),
    transform=val_transform
)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

print(f"✓ Train: {len(train_dataset)} | Val: {len(val_dataset)}")

## 6. Build U-Net Model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DoubleConv(nn.Module):
    """(Conv -> BN -> ReLU) * 2"""
    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )
    
    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""
    def __init__(self, in_channels, out_channels, bilinear=False):
        super().__init__()
        
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)
    
    def forward(self, x1, x2):
        x1 = self.up(x1)
        
        # Pad x1 to match x2 size if needed
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]
        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    """Output convolution"""
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)
    
    def forward(self, x):
        return self.conv(x)


class UNet(nn.Module):
    """
    Manual U-Net Implementation
    Standard architecture with 4 encoder and 4 decoder blocks
    """
    def __init__(self, n_channels=3, n_classes=6, bilinear=False):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear
        
        # Encoder
        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)
        
        # Decoder
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        
        # Output
        self.outc = OutConv(64, n_classes)
    
    def forward(self, x):
        # Encoder with skip connections
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        
        # Decoder with skip connections
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        
        logits = self.outc(x)
        return logits


# ============================================================================
# CREATE MODEL
# ============================================================================

# Create the model
model = UNet(n_channels=3, n_classes=NUM_CLASSES, bilinear=False)

# Move to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"✅ Manual U-Net created!")
print(f"   Architecture: Standard U-Net with skip connections")
print(f"   Input channels: 3 (RGB)")
print(f"   Output classes: {NUM_CLASSES}")
print(f"   Total parameters: {total_params:,}")
print(f"   Trainable parameters: {trainable_params:,}")
print(f"   Device: {device}")
print(f"   Model size: ~{total_params * 4 / (1024**2):.2f} MB")

# Test forward pass
test_input = torch.randn(1, 3, IMG_SIZE, IMG_SIZE).to(device)
with torch.no_grad():
    test_output = model(test_input)
print(f"\n✅ Forward pass test:")
print(f"   Input shape:  {list(test_input.shape)}")
print(f"   Output shape: {list(test_output.shape)}")
print(f"\n🎯 Model ready for training!")


# ============================================================================
# OPTIONAL: Model Architecture Visualization
# ============================================================================

def print_architecture(model):
    """Print detailed layer-by-layer architecture"""
    print("\n" + "="*60)
    print("DETAILED ARCHITECTURE")
    print("="*60)
    print("\n📥 ENCODER (Contracting Path):")
    print("  inc:   DoubleConv(3 → 64)")
    print("  down1: MaxPool → DoubleConv(64 → 128)")
    print("  down2: MaxPool → DoubleConv(128 → 256)")
    print("  down3: MaxPool → DoubleConv(256 → 512)")
    print("  down4: MaxPool → DoubleConv(512 → 1024)")
    
    print("\n📤 DECODER (Expansive Path):")
    print("  up1: ConvTranspose(1024 → 512) + Skip(512) → DoubleConv → 512")
    print("  up2: ConvTranspose(512 → 256) + Skip(256) → DoubleConv → 256")
    print("  up3: ConvTranspose(256 → 128) + Skip(128) → DoubleConv → 128")
    print("  up4: ConvTranspose(128 → 64) + Skip(64) → DoubleConv → 64")
    
    print("\n🎯 OUTPUT:")
    print(f"  outc: Conv1x1(64 → {model.n_classes})")
    print("="*60)

# Uncomment to see detailed architecture
# print_architecture(model)


# ============================================================================
# TIPS FOR TRAINING
# ============================================================================

print("\n" + "="*60)
print("💡 TRAINING TIPS")
print("="*60)
print("1. This is a standard U-Net from scratch (no pretrained weights)")
print("2. It may need more epochs to converge vs transfer learning")
print("3. Consider using:")
print("   - Data augmentation (already in your pipeline)")
print("   - Learning rate scheduling (already configured)")
print("   - Mixed precision training (already enabled)")
print("4. Monitor validation loss to avoid overfitting")
print("5. Expected model performance:")
print("   - Should reach ~0.60-0.75 mIoU on road damage dataset")
print("   - Training time: ~2-3 hours on T4 GPU for 60 epochs")
print("="*60)

## 7. Training Setup

In [ ]:
import torch.nn as nn
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau

class DiceLoss(nn.Module):
    def __init__(self, smooth=1.0):
        super().__init__()
        self.smooth = smooth
    
    def forward(self, pred, target):
        pred = torch.softmax(pred, dim=1)
        target_one_hot = torch.nn.functional.one_hot(target, NUM_CLASSES).permute(0, 3, 1, 2).float()
        intersection = (pred * target_one_hot).sum(dim=(2, 3))
        union = pred.sum(dim=(2, 3)) + target_one_hot.sum(dim=(2, 3))
        dice = (2.0 * intersection + self.smooth) / (union + self.smooth)
        return 1 - dice.mean()

ce_loss = nn.CrossEntropyLoss()
dice_loss = DiceLoss()

def combined_loss(pred, target):
    return ce_loss(pred, target) + dice_loss(pred, target)

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5, verbose=True)

print("✓ Loss, optimizer, scheduler ready")

## 8. Metrics Functions

In [ ]:
def calculate_iou(pred, target, num_classes):
    pred = torch.argmax(pred, dim=1)
    ious = []
    per_class_iou = []
    for cls in range(num_classes):
        pred_cls = (pred == cls)
        target_cls = (target == cls)
        intersection = (pred_cls & target_cls).sum().float()
        union = (pred_cls | target_cls).sum().float()
        if union > 0:
            iou = (intersection / union).item()
            ious.append(iou)
            per_class_iou.append(iou)
        else:
            per_class_iou.append(float('nan'))
    mean_iou = np.nanmean(per_class_iou) if per_class_iou else 0.0
    return mean_iou, per_class_iou

def calculate_pixel_accuracy(pred, target):
    pred = torch.argmax(pred, dim=1)
    correct = (pred == target).sum()
    total = target.numel()
    return (correct / total).item()

print("✓ Metrics ready")

## 9. Training Loop

In [ ]:
from tqdm import tqdm
import time

history = {'train_loss': [], 'val_loss': [], 'val_iou': [], 'val_accuracy': [], 'lr': [], 'val_per_class_iou': []}
best_iou = 0.0
patience_counter = 0
best_model_path = os.path.join(OUTPUT_DIR, 'best_unet_model.pth')
last_model_path = os.path.join(OUTPUT_DIR, 'last_unet_model.pth')

print("🚀 Starting training...\n")
start_time = time.time()

scaler = torch.cuda.amp.GradScaler() if torch.cuda.is_available() else None

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    
    # Training
    model.train()
    train_loss = 0.0
    for images, masks in tqdm(train_loader, desc='Training'):
        images, masks = images.to(device), masks.to(device)
        optimizer.zero_grad()
        if scaler:
            with torch.cuda.amp.autocast():
                outputs = model(images)
                loss = combined_loss(outputs, masks)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(images)
            loss = combined_loss(outputs, masks)
            loss.backward()
            optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)
    
    # Validation
    model.eval()
    val_loss = val_iou = val_accuracy = 0.0
    val_per_class_iou = np.zeros(NUM_CLASSES)
    val_batches = 0
    with torch.no_grad():
        for images, masks in tqdm(val_loader, desc='Validation'):
            images, masks = images.to(device), masks.to(device)
            if scaler:
                with torch.cuda.amp.autocast():
                    outputs = model(images)
                    val_loss += combined_loss(outputs, masks).item()
            else:
                outputs = model(images)
                val_loss += combined_loss(outputs, masks).item()
            mean_iou, per_class_iou = calculate_iou(outputs, masks, NUM_CLASSES)
            val_iou += mean_iou
            val_accuracy += calculate_pixel_accuracy(outputs, masks)
            val_per_class_iou += np.nan_to_num(per_class_iou)
            val_batches += 1
    
    val_loss /= len(val_loader)
    val_iou /= len(val_loader)
    val_accuracy /= len(val_loader)
    val_per_class_iou /= val_batches
    
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['val_iou'].append(val_iou)
    history['val_accuracy'].append(val_accuracy)
    history['lr'].append(optimizer.param_groups[0]['lr'])
    history['val_per_class_iou'].append(val_per_class_iou.tolist())
    
    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | mIoU: {val_iou:.4f} | Acc: {val_accuracy:.4f}")
    print(f"Per-class IoU: {val_per_class_iou}")
    
    scheduler.step(val_iou)
    
    # Save last model checkpoint
    torch.save({'model': model.state_dict(), 'iou': val_iou, 'history': history}, last_model_path)
    
    if val_iou > best_iou:
        best_iou = val_iou
        patience_counter = 0
        torch.save({'model': model.state_dict(), 'iou': best_iou, 'history': history}, best_model_path)
        print(f"✅ Best model saved! mIoU: {best_iou:.4f}")
    else:
        patience_counter += 1
        if patience_counter >= PATIENCE:
            print("Early stopping!")
            break

print(f"\n🎉 Training done in {(time.time()-start_time)/60:.1f} min | Best mIoU: {best_iou:.4f}")

## 10. Backup to Drive

In [ ]:
import shutil
drive_backup = "/content/drive/MyDrive/unet_models"
os.makedirs(drive_backup, exist_ok=True)
shutil.copy(best_model_path, os.path.join(drive_backup, 'unet_best.pth'))
print("✓ Backed up to Drive")

## 11. Training Curves

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

axes[0,0].plot(history['train_loss'], label='Train')
axes[0,0].plot(history['val_loss'], label='Val')
axes[0,0].set_title('Loss')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

axes[0,1].plot(history['val_iou'])
axes[0,1].axhline(best_iou, color='r', linestyle='--', label=f'Best: {best_iou:.4f}')
axes[0,1].set_title('mIoU')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

axes[1,0].plot(history['val_accuracy'])
axes[1,0].set_title('Accuracy')
axes[1,0].grid(True, alpha=0.3)

axes[1,1].plot(history['lr'])
axes[1,1].set_title('Learning Rate')
axes[1,1].set_yscale('log')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'training_curves.png'), dpi=150)
plt.show()

## 12. Test Set Evaluation

In [ ]:
checkpoint = torch.load(best_model_path)
model.load_state_dict(checkpoint['model'])
model.eval()

test_dataset = RoadDamageDataset(
    os.path.join(DATASET_ROOT, 'test', 'images'),
    os.path.join(DATASET_ROOT, 'test', 'masks'),
    transform=val_transform
)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

test_iou = test_acc = 0.0
with torch.no_grad():
    for images, masks in tqdm(test_loader, desc='Testing'):
        images, masks = images.to(device), masks.to(device)
        outputs = model(images)
        test_iou += calculate_iou(outputs, masks, NUM_CLASSES)
        test_acc += calculate_pixel_accuracy(outputs, masks)

test_iou /= len(test_loader)
test_acc /= len(test_loader)

print(f"\n📊 Test Results: mIoU={test_iou:.4f} | Accuracy={test_acc:.4f}")

## 13. Visualize Predictions

In [ ]:
import random

COLORS = np.array([[0,0,0], [255,0,0], [0,255,0], [0,0,255], [255,255,0], [255,0,255]], dtype=np.uint8)

test_imgs = os.listdir(os.path.join(DATASET_ROOT, 'test', 'images'))
samples = random.sample(test_imgs, min(6, len(test_imgs)))

fig, axes = plt.subplots(6, 3, figsize=(15, 24))

for idx, img_name in enumerate(samples):
    img_path = os.path.join(DATASET_ROOT, 'test', 'images', img_name)
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    mask_path = os.path.join(DATASET_ROOT, 'test', 'masks', os.path.splitext(img_name)[0] + '.png')
    gt_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    
    aug = val_transform(image=image)
    inp = aug['image'].unsqueeze(0).to(device)
    
    with torch.no_grad():
        pred = torch.argmax(model(inp), dim=1).cpu().numpy()[0]
    
    img_r = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    gt_r = cv2.resize(gt_mask, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_NEAREST)
    
    axes[idx,0].imshow(img_r)
    axes[idx,0].set_title(f'Original\n{img_name}')
    axes[idx,0].axis('off')
    
    axes[idx,1].imshow(COLORS[gt_r])
    axes[idx,1].set_title('Ground Truth')
    axes[idx,1].axis('off')
    
    axes[idx,2].imshow(COLORS[pred])
    axes[idx,2].set_title('Prediction')
    axes[idx,2].axis('off')

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'predictions.png'), dpi=150)
plt.show()